In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
# Datenimport
abstand = pd.DataFrame()
abstand['3cm'] = np.genfromtxt('data/IV_60_Alu.txt')
abstand['4cm'] = np.genfromtxt('data/VII_4cm.txt')
abstand['5cm'] = np.genfromtxt('data/VII_5cm.txt')
abstand['6cm'] = np.genfromtxt('data/VII_6cm.txt')
abstand['7cm'] = np.genfromtxt('data/VII_7cm.txt')
abstand['8cm'] = np.genfromtxt('data/VII_8cm.txt')
abstand['9cm'] = np.genfromtxt('data/VII_9cm.txt')
abstand['10cm'] = np.genfromtxt('data/VII_10cm.txt')
abstand['11cm'] = np.genfromtxt('data/VII_11cm.txt')
abstand['12cm'] = np.genfromtxt('data/VII_12cm.txt')

In [ ]:
# Zählrate bestimmen
abstand_rate = pd.DataFrame()
for i in abstand.columns:
    abstand_rate[i] = abstand[i].iloc[2:] / (abstand[i].iloc[0]-abstand[i].iloc[1])

In [ ]:
# Messunsicherheiten
abstand_n_delta = []
for i in abstand.columns:
    abstand_n_delta.append(np.sqrt(abstand[i].iloc[2:].sum())/ (abstand[i].iloc[0]-abstand[i].iloc[1]))
delta_n = np.asarray(abstand_n_delta)

In [ ]:
# Funktion zu fitten
def I(x, a, b, c):
    return a/(np.pi * 4 * (x-b)**2)+c

In [ ]:
i, i_std = curve_fit(I, np.linspace(3, 12, 10), abstand_rate.sum(), p0=[24,0,10], maxfev=1000)

In [ ]:
plt.errorbar(np.linspace(3, 12, 10), abstand_rate.sum(), fmt='x', xerr=0.2, yerr=delta_n, label='$\dot{N}_G$')
plt.plot(np.linspace(3, 12, 1000), [I(j, i[0], i[1], i[2]) for j in np.linspace(3, 12, 1000)], label='Fit')
plt.xlabel('Abstand Quelle Streukörper [cm]')
plt.ylabel('Zählrate [s⁻¹]')
plt.legend()
plt.savefig('images/abstand.png', dpi=300)

In [ ]:
abstand_rate.iloc[950:1100].plot()

In [ ]:
abstand_normiert = pd.DataFrame()
for i in abstand_rate.columns:
    abstand_normiert[i] = abstand_rate['3cm'].max() * abstand_rate[i] / abstand_rate[i].max()
abstand_normiert.iloc[950:1100].plot()

In [ ]:
def gaussian(x, a, x_0, sigma):
    return a*np.exp(-(x-x_0)**2/sigma**2)

In [ ]:
breite = []
breite_std = []
abstand_normiert['N'] = np.asarray(range(16382)) + 1
abstand_normiert.describe()
for i in abstand_normiert.drop(columns=['N']).columns:
    fit, std = curve_fit(gaussian, abstand_normiert['N'].iloc[980:1050], abstand_normiert[i].iloc[980:1050], p0=[0.3, 1010, 3])
    breite.append(fit[2])
    breite_std.append(np.sqrt(std[2][2]))

plt.errorbar(np.linspace(3,12,10), breite, fmt='x', yerr=breite_std, xerr=0.2)
plt.xlabel('Abstand Quelle Streukörper [cm]')
plt.ylabel('Peakbreite Kanal [-]')
plt.savefig('images/peakbreite.png', dpi=300)